In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:

spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [2,694 kB]
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [3,133 kB]
Get:14 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InR

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineReviewAnalysis").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Health_Personal_Care_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     650634| R3EQSTM9PWRAL|B0091LBZSU|     578484426|Demograss Capsule...|Health & Personal...|          3|            0|          0|   N|                Y|         Three Stars|Only came with 30...| 2015-08-31|
|         US|   19827510| RBWPRK17XKIXD|B00PWW3LQ6|     456433146|Viva Labs #1 Prem...|Health & Personal

In [7]:
# (1) filter DF for total votes > 20
total_df = df.filter(df.total_votes >= 20)
total_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1717317|R2984F9VD9WDJB|B0013OVVK0|     895291357|Jarrow Formulas N...|Health & Personal...|          1|           37|         46|   N|                Y|... month 2000 mg...|took it upon my d...| 2015-08-31|
|         US|   37405868|R218TGWCIAZYNY|B0065ZTKWS|     961894951|New Mama Bottom S...|Health & Personal

In [14]:
# (2) % of helpful votes to total votes
helpful_df = total_df.filter((total_df.helpful_votes / total_df.total_votes) >= 0.5)
helpful_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1717317|R2984F9VD9WDJB|B0013OVVK0|     895291357|Jarrow Formulas N...|Health & Personal...|          1|           37|         46|   N|                Y|... month 2000 mg...|took it upon my d...| 2015-08-31|
|         US|   37405868|R218TGWCIAZYNY|B0065ZTKWS|     961894951|New Mama Bottom S...|Health & Personal

In [16]:
# (3) add vine filter
vine_df = helpful_df.filter(helpful_df.vine =='Y')
vine_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   34746219| RG70K1HMY4LXX|B010H76PTU|     581545062|Omron 7 Series Wi...|Health & Personal...|          4|           37|         44|   Y|                N|Only tracks 1 per...|This product come...| 2015-08-27|
|         US|   51401810|R1DHGTNXDXJ0GB|B00VAPRSP4|     373707856|Cellucor, SuperHD...|Health & Personal

In [17]:
# (3) add vine filter
nonvine_df = helpful_df.filter(helpful_df.vine =='N')
nonvine_df.show(10)

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    1717317|R2984F9VD9WDJB|B0013OVVK0|     895291357|Jarrow Formulas N...|Health & Personal...|          1|           37|         46|   N|                Y|... month 2000 mg...|took it upon my d...| 2015-08-31|
|         US|   37405868|R218TGWCIAZYNY|B0065ZTKWS|     961894951|New Mama Bottom S...|Health & Personal

In [18]:
# Number of paid reviews
paid_reviews = vine_df.count()
paid_reviews

497

In [20]:
# Number of paid 5 start
paid_five = vine_df.filter(vine_df.star_rating ==5).count()
paid_five

220

In [21]:
# Paid 5 star review %
paid_five_perc = (paid_five/paid_reviews)*100
paid_five_perc


44.26559356136821

In [22]:
# Number of unpaid reviews
unpaid_reviews = nonvine_df.count()
unpaid_reviews

120863

In [23]:
# Number of unpaid 5 start
unpaid_five = nonvine_df.filter(nonvine_df.star_rating ==5).count()
unpaid_five

74470

In [24]:
# Unpaid 5 star review %
unpaid_five_perc = (unpaid_five/unpaid_reviews)*100
unpaid_five_perc


61.615217229425056

In [26]:
print(paid_reviews)
print(unpaid_reviews)

497
120863


In [27]:
print(paid_five)
print(unpaid_five)

220
74470


In [28]:
print(paid_five_perc)
print(unpaid_five_perc)

44.26559356136821
61.615217229425056
